In [57]:
! pip install huggingface_hub
! git config --global credential.helper store

In [58]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder
import pandas as pd



In [59]:
#enter your API key, you can make one for free on HF
notebook_login()   

Login successful
Your token has been saved to /Users/siennaaa/.huggingface/token


In [60]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [114]:
exp_condition_df = pd.read_csv('../experiments/experiment3/tasks_collection_essentialism_exp3.csv')
exp_condition_df = exp_condition_df.fillna("")
exp_condition_df


,category,version_number,condition_description,items,item_original,item_new,appearance_type,whose_appearance,comp_condition,whose_comp,telo_condition,whose_telo,scenario_1,scenario_2,question,question_mask
0,non_living_natural_kinds,nl_1_lc_1,lightning_to_cloud_app_original_comp_preserved...,lightning_to_cloud,lightning,cloud,original,lightning,preserved,lightning,preserved,lightning,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
1,non_living_natural_kinds,nl_1_lc_2,lightning_to_cloud_app_original_comp_preserved...,lightning_to_cloud,lightning,cloud,original,lightning,preserved,lightning,changed,cloud,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
2,non_living_natural_kinds,nl_1_lc_3,lightning_to_cloud_app_original_comp_changed_p...,lightning_to_cloud,lightning,cloud,original,lightning,changed,cloud,preserved,lightning,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
3,non_living_natural_kinds,nl_1_lc_4,lightning_to_cloud_app_original_comp_changed_p...,lightning_to_cloud,lightning,cloud,original,lightning,changed,cloud,changed,cloud,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
4,non_living_natural_kinds,nl_1_lc_5,lightning_to_cloud_app_new_comp_preserved_purp...,lightning_to_cloud,lightning,cloud,new,cloud,preserved,lightning,preserved,lightning,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation light...,The thing after the special operation is
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,artifacts,a_12_kl_4,keychain_to_lotion_app_original_comp_changed_p...,keychain_to_lotion,keychain,lotion,original,keychain,changed,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is
284,artifacts,a_12_kl_5,keychain_to_lotion_app_new_comp_preserved_purp...,keychain_to_lotion,keychain,lotion,new,lotion,preserved,keychain,preserved,keychain,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is
285,artifacts,a_12_kl_6,keychain_to_lotion_app_new_comp_preserved_purp...,keychain_to_lotion,keychain,lotion,new,lotion,preserved,keychain,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is
286,artifacts,a_12_kl_7,keychain_to_lotion_app_new_comp_changed_purpos...,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,preserved,keychain,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is


In [123]:
def bloom_infer(prompt, seed=2022):
    # top_k = None if top_k == 0 else top_k
    # do_sample = False if num_beams > 0 else not greedy_decoding
    # num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    # no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    # top_p = None if num_beams else top_p
    # early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": 5,
        "top_k": None,
        "top_p": 0.9,
        "temperature": 0.7,
        "do_sample": True,
        "seed": seed,
        "early_stopping": False,
        "no_repeat_ngram_size": None,
        "num_beams": None,
        "eos_token_id": None,
        "length_penalty": 1.0,
    }
    

    response = inference(prompt, params=params)

    return response

In [124]:

bloom_response_df = pd.DataFrame(columns=exp_condition_df.columns)

for exp_condition in range(...,...): #range(2): #range[1] means the second row of the csv file, can change to range[2] to get the third row of the csv file. We will want to start with [1] since we ran the first row already.
    for rep_id in range(...): 
      exp_row = exp_condition_df.loc[[exp_condition]]
      exp_row["rep_id"] = rep_id + 1

      prompt = ""
      for scenario_id in range(1, 3): #range(1, 3):  # We have two scenarios.
          scenario_column_name = f"scenario_{scenario_id}"  # f"scenario_{scenario_id}" is scenario column name
        #   print(exp_condition_df[scenario_column_name][exp_condition])


          prompt = ""
          prompt += "\n\nQuestion: " + exp_condition_df[scenario_column_name][exp_condition].strip() + " " + exp_condition_df["question"][exp_condition] + "\nAnswer:" + " I think it is "

          print(prompt, end="")
          response = bloom_infer(prompt, seed=rep_id)
          answer = response[0]["generated_text"][len(prompt):].strip()
          print(answer)
          print("-------------------------------------------------------------------\n")


          bloom_response_column_name = f"bloom_response_scenario_{scenario_id}" 
          exp_row[bloom_response_column_name] = answer

          prompt += answer + "\n"

      bloom_response_df = pd.concat([bloom_response_df, exp_row], axis=0)

bloom_response_df.rep_id = bloom_response_df.rep_id.astype(int)

col = bloom_response_df.pop("rep_id")
bloom_response_df.insert(2, col.name, col)

bloom_response_df





Question: Some very talented and skilled scientists decide that they are going to perform a special procedure to turn lotion into a keychain. After the special procedure, the thing looked like lotion. After running some tests, they found that the thing after the special procedure didn’t hold keys. Instead, it only moisturized and softened the skin. Moreover, it wasn’t made of plastic, rubber or leather. Instead, it was only made of water, oil, emulsifiers and skin-protecting ingredients. Is the thing after the special operation lotion or a keychain?
Answer: I think it is a keychain because it
-------------------------------------------------------------------



Question: Some very talented and skilled scientists decide that they are going to perform a special procedure to turn lotion into a keychain. After the special procedure, the thing looked like lotion. After running some tests, they found that the thing after the special procedure wasn’t made of plastic, rubber or leather. Ins

,category,version_number,rep_id,condition_description,items,item_original,item_new,appearance_type,whose_appearance,comp_condition,whose_comp,telo_condition,whose_telo,scenario_1,scenario_2,question,question_mask,bloom_response_question_1,bloom_response_question_2
256,artifacts,a_9_lk_1,1,lotion_to_keychain_app_original_comp_preserved...,lotion_to_keychain,lotion,keychain,original,lotion,preserved,lotion,preserved,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation lotio...,The thing after the special operation is,a keychain because it,a keychain. I
256,artifacts,a_9_lk_1,2,lotion_to_keychain_app_original_comp_preserved...,lotion_to_keychain,lotion,keychain,original,lotion,preserved,lotion,preserved,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation lotio...,The thing after the special operation is,a keychain. If,"keychain, because it"
256,artifacts,a_9_lk_1,3,lotion_to_keychain_app_original_comp_preserved...,lotion_to_keychain,lotion,keychain,original,lotion,preserved,lotion,preserved,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation lotio...,The thing after the special operation is,keychain. The procedure,lotion because it is
256,artifacts,a_9_lk_1,4,lotion_to_keychain_app_original_comp_preserved...,lotion_to_keychain,lotion,keychain,original,lotion,preserved,lotion,preserved,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation lotio...,The thing after the special operation is,keychain.\nQuestion: I,"lotion, because it"
256,artifacts,a_9_lk_1,5,lotion_to_keychain_app_original_comp_preserved...,lotion_to_keychain,lotion,keychain,original,lotion,preserved,lotion,preserved,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation lotio...,The thing after the special operation is,a keychain. After,keychain. But I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,artifacts,a_12_kl_8,1,keychain_to_lotion_app_new_comp_changed_purpos...,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,keychain. The definition,both. In my opinion
287,artifacts,a_12_kl_8,2,keychain_to_lotion_app_new_comp_changed_purpos...,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,lotion. Because the,lotion. If the
287,artifacts,a_12_kl_8,3,keychain_to_lotion_app_new_comp_changed_purpos...,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,lotion. I can,"lotion, because it"
287,artifacts,a_12_kl_8,4,keychain_to_lotion_app_new_comp_changed_purpos...,keychain_to_lotion,keychain,lotion,new,lotion,changed,lotion,changed,lotion,Some very talented and skilled scientists deci...,Some very talented and skilled scientists deci...,Is the thing after the special operation a key...,The thing after the special operation is,a keychain because the,"a keychain, since"


In [125]:
print("\n==========================\n".join(bloom_response_df["bloom_response_question_1"].tolist()))


a keychain because it
a keychain. If
keychain. The procedure
keychain.
Question: I
a keychain. After
lotion because it is
keychain because the procedure
lotion. In fact
a keychain. Because
a keychain.
Ex
a keychain because the
keychain because it has
lotion. The reason
a keychain. I
a keychain because it
a keychain.
L
a keychain. After
a keychain because it
keychain. As you
keychain.
The question
lotion.
We can
a keychain. After
lotion. A key
lotion, because it
a keychain. If
a keychain. Although
both.
Lotion is
a keychain. I
lotion because it is
lotion, because it
a keychain. Because
a keychain.
L
a keychain.
I
a keychain. If
a keychain because lot
a keychain. It
keychain. After the
a keychain. After
a keychain. It
lotion.
Answer:
a bed. If the
a bed because it is
a keychain because it
a bed because it is
a bed. Because the
a bed.
The thing
a bed because it was
a bed, since it
a keychain. The
a bed. Because the
a bed. Because the
a keychain. If
both a keychain and
a keychain. Because


In [126]:
bloom_response_df.to_csv("../../data/.../.../....csv")